In [34]:
#import necessary packages

import json
import numpy as np
import pandas as pd
import datasets
from datasets.dataset_dict import DatasetDict
import itertools
import random
from typing import List
from sklearn import metrics
from sklearn.metrics import f1_score

# import matplotlib.pyplot as plt
from tqdm import tqdm
import torch,csv
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification,
    TrainingArguments, 
    Trainer
)
import evaluate
import numpy as np

metric = evaluate.load("accuracy")


In [35]:
#select device GPU, CPU or any other

if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

In [36]:
#Data Statistics
d=pd.read_csv("data/train.csv")
d["label"].value_counts()

label
neutral     1266
posiitve     853
negative     363
Name: count, dtype: int64

In [37]:
#Data Statistics
d=pd.read_csv("data/dev.csv")
d["label"].value_counts()

label
neutral     305
posiitve    219
negative     97
Name: count, dtype: int64

In [38]:
#Data Statistics
d=pd.read_csv("data/test.csv")
d["label"].value_counts()

label
neutral     386
posiitve    280
negative    110
Name: count, dtype: int64

In [39]:
#read csv and return list of instances

def read_data(path, to_dataset=True):
    '''
    read csv and return list of instances
    '''
    rows = []; label_dict = {'negative': 0, 'neutral': 1, 'positive': 2}

    
    # read data
    df=pd.read_csv(path)
    for i in range(len(df)):
        text = df['tweet'].loc[i]
        
        #remove extra space
        wrds = text.split(" ")
        new_wrds = [wrd.strip() for wrd in wrds]
        text = " ".join(new_wrds)
        
        label = df['label'].loc[i]
        if(label in ["posiitve"]):
            label="positive"
            
        label = label_dict[label]
        
        
        rows.append({
            'id': i,
            'text': text,
            'label': label,
        })
        
  
    if to_dataset:
        # to object
        rows = datasets.Dataset.from_list(rows)

    return rows

In [40]:
def compute_metrics(eval_pred):
    '''
    calculate accuracy
    '''
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [41]:
class BERT:
    '''
    with tokenizer and auto model
    '''
    def __init__(self, MODEL, num_labels):
        '''
        basic
        '''
        self.tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-multilingual-uncased")
        self.model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels)
    
    def tokenize(self, samples):
        '''
        run tokenization
        '''
        return self.tokenizer(samples["text"], padding="max_length", max_length=50, truncation=True)
    
    def run(self, sample):
        '''
        run for a single instance
        '''
        encoded_input = self.tokenizer(sample, return_tensors='pt')
        # print("encoded")
        output = self.model(**encoded_input)['logits'][0].detach().numpy()

        return np.argmax(output)
    

In [42]:
MODEL = "google-bert/bert-base-multilingual-uncased"

# init model
model_module = BERT(MODEL, num_labels=3)

# read dataset
train_dataset = read_data('data/train.csv')
dev_dataset = read_data('data/dev.csv')

dataset = DatasetDict(
    {
        'train': train_dataset, 
        'dev': dev_dataset, 
    })


/opt/anaconda3/envs/robust/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-multilingual-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 2482
    })
    dev: Dataset({
        features: ['id', 'text', 'label'],
        num_rows: 621
    })
})

In [44]:
# tokenize dataset (multiple examples at same time)
tok_dataset = dataset.map(model_module.tokenize, batched=True)

Map:   0%|          | 0/2482 [00:00<?, ? examples/s]

Map:   0%|          | 0/621 [00:00<?, ? examples/s]

In [45]:
'''
input_ids
These are the numerical representations of tokens from the model’s vocabulary.
Each token is mapped to a unique ID.
[101] → [CLS] (Start of sentence)
102 → [SEP] (End of sentence)

attention_mask
Specifies which tokens should be attended to and which should be ignored (useful for padding).
1 means the token is real, 0 means it's padding.

token_type_ids (Segment IDs)
Helps differentiate between two sequences in a single input.
'''
tok_dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2482
    })
    dev: Dataset({
        features: ['id', 'text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 621
    })
})

In [46]:
print(tok_dataset["train"][1]["text"])
print(model_module.tokenizer.convert_ids_to_tokens(tok_dataset["train"][1]["input_ids"]))
print(tok_dataset["train"][1]["input_ids"])
print(tok_dataset["train"][1]["token_type_ids"])
print(tok_dataset["train"][1]["attention_mask"])


c0mment scroll kar kar ke hairan hogaye hum
['[CLS]', 'c', '##0', '##mment', 'sc', '##roll', 'kar', 'kar', 'ke', 'hair', '##an', 'hog', '##aye', 'hum', '[SEP]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]', '[PAD]']
[101, 145, 10995, 46885, 16427, 70066, 21185, 21185, 11009, 29708, 10208, 57894, 48634, 52179, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [31]:
# training
# training_args = TrainingArguments(output_dir="./logs", num_train_epochs=1, learning_rate=3e-05, eval_strategy="epoch")

training_args = TrainingArguments(
    output_dir="./logs_mbert",           # Where model checkpoints/logs will be saved
    num_train_epochs=3,            # Number of training epochs
    learning_rate=3e-05,           # Learning rate (3e-5 = 0.00003)
    save_strategy="epoch",
    evaluation_strategy="epoch",    # Evaluate the model at the end of each epoch
    per_device_train_batch_size=8,  # Training batch size
    per_device_eval_batch_size=8, # Eval batch size
    load_best_model_at_end=True, # Load best model when training ends
    metric_for_best_model="accuracy",
    greater_is_better=True
)


In [32]:
trainer = Trainer(
    model=model_module.model,
    args=training_args,
    train_dataset=tok_dataset["train"],
    eval_dataset=tok_dataset["dev"],
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.847369,0.619968
2,0.832400,0.720731,0.702093
3,0.832400,0.792255,0.711755


TrainOutput(global_step=933, training_loss=0.694607913557995, metrics={'train_runtime': 364.5996, 'train_samples_per_second': 20.422, 'train_steps_per_second': 2.559, 'total_flos': 191322510586200.0, 'train_loss': 0.694607913557995, 'epoch': 3.0})

In [48]:
#Inferencing

# dataset
rows = read_data("data/test.csv", to_dataset=False)

#Load model 
MODEL = "logs_mbert/"

model_module = BERT(MODEL, num_labels=3)

/opt/anaconda3/envs/robust/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [49]:
pred = []; gold = []
for row in tqdm(rows, total=len(rows)):
    pred.append(model_module.run(row['text'])); gold.append(row['label'])
  

100%|█████████████████████████████████████████| 776/776 [00:31<00:00, 24.75it/s]


In [50]:
accuracy = np.sum(np.array(pred) == np.array(gold)) / len(gold)
print("ACCURACY:", accuracy)

# f1 score
for mode in ['macro']:
    f1 = f1_score(gold, pred, average=mode)
    print(f"F1: {f1}")
print('-------------------------')

# confusion matrix
print(metrics.confusion_matrix(gold, pred))

ACCURACY: 0.6739690721649485
F1: 0.6398951987641559
-------------------------
[[ 58  38  14]
 [ 28 287  71]
 [ 18  84 178]]


Evaluating Robustness of Sentiment Classifier 

In [56]:
#generate phonetic perturbations
class PhoneticPerturbations():
    
    def __init__(
        self, seed: int = 0):
        
        self.seed=seed
        
    
    def pho_50(self, sentence: str):
        random.seed(self.seed)
        sentence_list = sentence.split()
        l=len(sentence_list)
        k_fifty=int(l/2)
       
        
        if(k_fifty==0):
            k_fifty=1
       
        perturbed_texts = []
        subwrd_dict={"aa": "a", "i": "ee", "r": "ri", "o": "oo", "au": "ou", "ka": "k", "kha": "kh", "ga": "g", 
				"gha": "gh", "ca": "c","cha": "ch", "sa":"sh", "jha": "jh", "bha":"v", "ta": "t", "tha": "th", "da": "d", "dha": "dh", "na":"n", 
				"pa": "p", "pha":"f", "ba": "b", "ma": "m", "ya":"y", "ra": "rh", "la": "l", "ja": "z", "ha": "h"}
        sub_keys=subwrd_dict.keys();sub_wrds=subwrd_dict.values()
        all_roots=list(subwrd_dict.keys())
               
        # Perturb the input sentence 
        cnt=1
        for idx, word in enumerate(sentence_list):
            
            if(cnt>k_fifty):
                break
               
            else:
                for j in range(len(all_roots)):
                    # print(all_roots[j])
                    if(all_roots[j] in word):
                        word=word.replace(all_roots[j], subwrd_dict[all_roots[j]])
                    
                sentence_list[idx]=word
                cnt=cnt+1
                

        perturbed_texts=" ".join(sentence_list)

        return perturbed_texts

    def pho_25(self, sentence: str):
        random.seed(self.seed)
        sentence_list = sentence.split()
        l=len(sentence_list)
        k_twentyfive=int(l/4)
       
        
        if(k_twentyfive==0):
            k_twentyfive=1
       
        perturbed_texts = []
        subwrd_dict={"aa": "a", "i": "ee", "r": "ri", "o": "oo", "au": "ou", "ka": "k", "kha": "kh", "ga": "g", 
				"gha": "gh", "ca": "c","cha": "ch", "sa":"sh", "jha": "jh", "bha":"v", "ta": "t", "tha": "th", "da": "d", "dha": "dh", "na":"n", 
				"pa": "p", "pha":"f", "ba": "b", "ma": "m", "ya":"y", "ra": "rh", "la": "l", "ja": "z", "ha": "h"}
        
        sub_keys=subwrd_dict.keys();sub_wrds=subwrd_dict.values()
        all_roots=list(subwrd_dict.keys())
               
        # Perturb the input sentence 
        cnt=1
        for idx, word in enumerate(sentence_list):
            
            if(cnt>k_twentyfive):
                break
               
            else:
                for j in range(len(all_roots)):
                    # print(all_roots[j])
                    if(all_roots[j] in word):
                        word=word.replace(all_roots[j], subwrd_dict[all_roots[j]])
                    
                sentence_list[idx]=word
                cnt=cnt+1
                

        perturbed_texts=" ".join(sentence_list)

        return perturbed_texts

In [57]:
pp=PhoneticPerturbations()

In [58]:
#Read test file and apply perturbations

df=pd.read_csv("data/test.csv")
print(len(df), df.columns)

adv_text_50=[];adv_text_25=[]
for i in range(len(df)):
    t=pp.pho_50(df.tweet.loc[i])
    t1=pp.pho_25(df.tweet.loc[i])
    
    adv_text_50.append(t)
    adv_text_25.append(t1)
    
df["adv_text_50"] = adv_text_50
df["adv_text_25"] = adv_text_25

print(df.columns)
df.to_csv("data/adv_test_set.csv")

776 Index(['Unnamed: 0', 'tweet', 'label'], dtype='object')
Index(['Unnamed: 0', 'tweet', 'label', 'adv_text_50', 'adv_text_25'], dtype='object')


In [59]:
print(df["tweet"][0])
print(df["adv_text_25"][0])
print(df["adv_text_50"][0])

Kon kareaga rea terea sath bt tujhsea acha dog sea bt kr lu
Koon krieag riea terea sath bt tujhsea acha dog sea bt kr lu
Koon krieag riea teriea shth bt tujhsea acha dog sea bt kr lu


In [60]:
#evaluate the model on perturbed dataset#
label_dict1 = {'negative': 0, 'neutral': 1, 'positive': 2}

def read_adv_data(path):
    '''
    read adv csv
    '''
    dataset = {}
    cnt=0
    with open(path, mode ='r') as f:
        data_f = csv.reader(f)
        # skip header
        next(data_f)

        for line in data_f:
            cnt=cnt+1
            # print("hiiiiiii",cnt,line)
            # label = label_dict1[line[-1]]
            label = line[3]

            dataset[str(line[0])] = {
                'adv_text_25': line[4].strip(),
                'adv_text_50': line[3].strip(),
                'label': label
            }
    return dataset

In [62]:
# dataset
rows = read_data("data/test.csv", to_dataset=False)

# load adv dataset
adv_rows = read_adv_data("data/adv_test_set.csv")

print("len", len(rows), len(adv_rows))

MODEL = "./logs_mbert"
# init model
model_module = BERT(MODEL, num_labels=3)


len 776 776


/opt/anaconda3/envs/robust/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [63]:
# 0.25 perturbation ratio

adv_check = 0; adv_success = 0

pred = []; adv_pred = []; gold = []; cnt = 0
for row in tqdm(rows, total=len(rows)):
    cnt = cnt+1

    predicted = model_module.run(row['text']) 
    label = row['label']
    # print(predicted,label)
    
    if predicted == label:
        adv_check += 1
        # rerun with adv claim
        # print(adv_rows[str(row['id'])]['adv_text'])
        adv_predicted = model_module.run(adv_rows[str(row['id'])]['adv_text_25'])
        # print("adv_predicted", adv_predicted, predicted)
        if adv_predicted != predicted:
            # on success
            adv_success += 1
    else:
        adv_predicted = predicted
    

    pred.append(predicted); adv_pred.append(adv_predicted); gold.append(label)


for y_pred, y_gold in zip([pred, adv_pred], [gold, gold]):
    # for a pair
    accuracy = np.sum(np.array(y_pred) == np.array(y_gold)) / len(y_gold)
    print("ACCURACY:", accuracy)
    # f1 score
    for mode in ['macro']:
        f1 = f1_score(y_gold, y_pred, average=mode)
        print(f"{mode.upper()} F1: {f1}")
    print('-------------------------\n')

    # confusion matrix
    print(metrics.confusion_matrix(y_gold, y_pred), '\n')
# print("**********************",len(pre))

print(f"adv check: {adv_check}, adv success: {adv_success}")
##
print(f"adv success rate: {(adv_success / adv_check) * 100}")


100%|█████████████████████████████████████████| 776/776 [00:49<00:00, 15.57it/s]

ACCURACY: 0.6739690721649485
MACRO F1: 0.6398951987641559
-------------------------

[[ 58  38  14]
 [ 28 287  71]
 [ 18  84 178]] 

ACCURACY: 0.5657216494845361
MACRO F1: 0.5170279768539707
-------------------------

[[ 43  52  15]
 [ 40 266  80]
 [ 25 125 130]] 

adv check: 523, adv success: 84
adv success rate: 16.061185468451242


In [66]:
# #0.50 perturbations

# adv_check = 0; adv_success = 0

# pred = []; adv_pred = []; gold = []; cnt = 0
# for row in tqdm(rows, total=len(rows)):
#     cnt = cnt+1

#     predicted = model_module.run(row['text']) 
#     label = row['label']
#     # print(predicted)
    
#     if predicted == label:
#         adv_check += 1
#         # rerun with adv claim
#         # print(adv_rows[str(row['id'])]['adv_text'])
#         adv_predicted = model_module.run(adv_rows[str(row['id'])]['adv_text_50'])
#         # print("adv_predicted", adv_predicted, predicted)
#         if adv_predicted != predicted:
#             # on success
#             adv_success += 1
#     else:
#         adv_predicted = predicted
    

#     pred.append(predicted); adv_pred.append(adv_predicted); gold.append(label)


# for y_pred, y_gold in zip([pred, adv_pred], [gold, gold]):
#     # for a pair
#     accuracy = np.sum(np.array(y_pred) == np.array(y_gold)) / len(y_gold)
#     print("ACCURACY:", accuracy)
#     # f1 score
#     for mode in ['macro']:
#         f1 = f1_score(y_gold, y_pred, average=mode)
#         print(f"{mode.upper()} F1: {f1}")
#     print('-------------------------\n')

#     # confusion matrix
#     print(metrics.confusion_matrix(y_gold, y_pred), '\n')
# # print("**********************",len(pre))

# print(f"adv check: {adv_check}, adv success: {adv_success}")
# ##
# print(f"adv success rate: {(adv_success / adv_check) * 100}")